In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow import keras
from keras import models
from keras.layers import Input, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
anime_df = pd.read_csv('/Users/zahidlaguna/Downloads/anime/anime_with_synopsis.csv')
ratings = pd.read_csv('/Users/zahidlaguna/Downloads/anime/rating_complete.csv')
anime_df 

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


In [4]:
anime_df = anime_df.replace('Unknown', np.nan)
anime_df = anime_df.dropna()

In [5]:
print(ratings.shape)
ratings

(57633278, 3)


,user_id,anime_id,rating
0,0,430,9
1,0,1004,5
2,0,3010,7
3,0,570,7
4,0,2762,9
...,...,...,...
57633273,353404,502,8
57633274,353404,987,4
57633275,353404,225,8
57633276,353404,243,7


In [6]:
#calculate the average rating of each anime
anime_rating = np.mean(ratings['rating'].groupby(ratings['anime_id']).mean())
anime_rating = np.ceil(anime_rating*10)/10
print('The Average anime rating', anime_rating)
print('The average rating of Dragon Ball Z is 7.6')

The Average anime rating 6.2
The average rating of Dragon Ball Z is 7.6


In [7]:
anime_df['content'] = anime_df['Name'] + ' ' + anime_df['Genres']
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(anime_df['content'])

In [8]:
scaler = MinMaxScaler()
X_norm = scaler.fit_transform(X.toarray())
cosine_sim = cosine_similarity(X_norm)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_norm, X_norm, test_size=0.1, random_state=42)
input_layer = Input(shape=(X_train.shape[1],))
hidden_layer1 = Dense(256, activation='relu')(input_layer)
dropout_layer1 = Dropout(0.2)(hidden_layer1)
hidden_layer2 = Dense(128, activation='relu')(dropout_layer1)
dropout_layer2 = Dropout(0.2)(hidden_layer2)
output_layer = Dense(X_norm.shape[1], activation='relu')(dropout_layer2) 

In [10]:
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, X_train, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
281/281 [==============================] - 3s 9ms/step - loss: 3.7917e-04 - val_loss: 3.2470e-04
Epoch 2/50
281/281 [==============================] - 2s 8ms/step - loss: 3.1171e-04 - val_loss: 2.9026e-04
Epoch 3/50
281/281 [==============================] - 2s 9ms/step - loss: 2.9226e-04 - val_loss: 2.7900e-04
Epoch 4/50
281/281 [==============================] - 2s 8ms/step - loss: 2.8424e-04 - val_loss: 2.7461e-04
Epoch 5/50
281/281 [==============================] - 2s 8ms/step - loss: 2.8021e-04 - val_loss: 2.7160e-04
Epoch 6/50
281/281 [==============================] - 2s 8ms/step - loss: 2.7707e-04 - val_loss: 2.6990e-04
Epoch 7/50
281/281 [==============================] - 2s 8ms/step - loss: 2.7462e-04 - val_loss: 2.6783e-04
Epoch 8/50
281/281 [==============================] - 2s 9ms/step - loss: 2.7274e-04 - val_loss: 2.6706e-04
Epoch 9/50
281/281 [==============================] - 2s 8ms/step - loss: 2.7127e-04 - val_loss: 2.6587e-04
Epoch 10/50
281/281 [=======

In [15]:
def get_input_vector(idx, norms):
   return norms[idx].reshape(1, -1) 
def recommend(anime_title, model, norms, indices, df, top_n=10):
   idx = indices[df['Name'] == anime_title].tolist()[0]
   anime_embedding = model.predict(get_input_vector(idx, norms))     
   sim_scores = cosine_similarity(anime_embedding, norms).reshape(-1)
   best_scores = np.argsort(sim_scores)[-top_n:]
   return df.iloc[best_scores]

In [21]:
anime_title = input("Enter an anime title: ")
recommendations = recommend(anime_title, model, X_norm, anime_df.index, anime_df)
if recommendations.empty:
    print("No recommendations found for {}".format(anime_title))
else: 
    print("Recommended anime similar to {}:".format(anime_title))
    print(recommendations)

1/1 [==============================] - 0s 15ms/step
Recommended anime similar to Naruto:
       MAL_ID                      Name Score  \
13320   37885  Super Dragon Ball Heroes  5.29   
8831    28755  Boruto: Naruto the Movie   7.5   
2592     3154                 Gatchaman  6.42   
3786     6033           Dragon Ball Kai  7.73   
5281    10075               Naruto x UT   7.4   
1508     1735        Naruto: Shippuuden  8.16   
192       223               Dragon Ball   8.0   
9560    30694         Dragon Ball Super  7.42   
716       813             Dragon Ball Z  8.16   
10         20                    Naruto  7.91   

                                                  Genres  \
13320  Action, Comedy, Super Power, Martial Arts, Fan...   
8831   Action, Comedy, Martial Arts, Shounen, Super P...   
2592   Action, Adventure, Martial Arts, Sci-Fi, Shoun...   
3786   Action, Adventure, Comedy, Fantasy, Martial Ar...   
5281   Action, Comedy, Super Power, Martial Arts, Sho...   
1508   Acti